Title: A Picard-Fuchs Algorithm for Hyperelliptic Level Curves.

In [ ]:
Author: Bradley Klee

In [ ]:
License: GPLv3. 

In [1]:
\\Input: Polynomial potential function V(x)
\\Output: One-form reduction matrices
{HyperEllipticData(poly)=
my(deg,Gx,GL,GR,ID,InvG);
deg=poldegree(poly,x);
Gz=matrix(2*deg-1,2*deg-1,j,k,if(j==k&&k<deg,z,0));
GL=matrix(2*deg-1,2*deg-1,j,k,if(0<j-k&&j-k<=deg&&k<deg,-2*polcoef(poly,j-k),0));
GR=matrix(2*deg-1,2*deg-1,j,k,if(k>=deg&&j-(k-deg)>0&&j-(k-deg)<=deg,-(j-(k-deg))*polcoef(poly,j-(k-deg)),0));
ID=matrix(2*deg-1,2*deg-1,j,k,if(j==k,1,0));
InvG=matinverseimage(Gz+GL+GR,ID);
MU=matrix(deg-1,deg-1,j,k,InvG[j,k]);
MV=matrix(deg,deg-1,j,k,InvG[j+deg-1,k]);
dMV=matrix(deg-1,deg-1,j,k,j*MV[j+1,k]);
[deg,MU,MV,dMV]};

In [2]:
\\Input: One-form and One-form reduction matrices
\\Output: Certified, hermite-reduced one form 
{HermiteReduce(dtform,cert,dat,n)=if(n==0,[dtform,cert],
HermiteReduce((dat[2]+1/(2*n-1)*dat[4])*dtform,
cert+1/((2*n-1)*y^(2*n-1))*dat[3]*dtform,
dat,n-1))};

In [3]:
\\n!!
{DoubleFactorial(n) = prod(i=0, (n-1)\2, n - 2*i )};

In [4]:
\\Input: Parity Boolean, one-form reduction matrices
\\Output: Raw Picard-Fuchs data
{PeriodBasis(sym,dat)=
my(d,dt,bound,reduction);
d=dat[1];
dt=vector(d-1,j,if(j==1,1,0))~;
bound=2*if(sym==0,floor((d-1)/2),floor(d/4));
decomposition=concat([[dt,vector(dat[1],j,0)~]],
vector(bound,j,HermiteReduce(((-1/2)^j)*DoubleFactorial(2*j-1)*dt,0,dat,j)));
[vector(bound+1,j,decomposition[j][1]),
vector(bound+1,j,vector(dat[1],j,x^(j-1))*decomposition[j][2]),
concat([1],vector(bound,j,((-1/(2*y^2))^(j)*DoubleFactorial(2*j-1))))]};

In [5]:
\\Input: Polynomial Potential Function V(x)
\\Output: Processed Picard-Fuchs data
{HyperellipticPicardFuchs(poly)=
my(HEDat,sym,decomposition,PFCs);
sym=if(subst(poly,x,-x)-poly==0,1,0);
HEDat = HyperEllipticData(poly);
decomposition = PeriodBasis(sym,HEDat);
PFCs=lindep(decomposition[1]);
[(1/2)*y^2+poly,decomposition[3]*dt,PFCs,decomposition[2]*PFCs]};

In [6]:
\\Input: Processed Picard-Fuchs data
\\Output: Boolean check value 0
{CheckCertificate(PFData)=
y^(2*length(PFData[2])-2)*(
deriv(PFData[4],x)*deriv(PFData[1],y)
+deriv(PFData[4],y)*deriv(-PFData[1],x)
+PFData[2]*PFData[3]/dt)%(2*PFData[1]-z)
};

In [7]:
\\(Cf. OEIS: A113424 )
PFData = HyperellipticPicardFuchs((1/2)*(x^2-(1/4)*x^4));
[PFData[3],CheckCertificate(PFData)]

[[3, 32*z - 16, 16*z^2 - 16*z]~, 0]

In [8]:
\\(Cf. OEIS: A000897 )
PFData = HyperellipticPicardFuchs((1/2)*(x^2-c*x^3));
[-vector(3,j,9/4*(polcoef(PFData[3][j],2,c)*4/27+polcoef(PFData[3][j],0,c))),CheckCertificate(PFData)]

[[-5, -72*z + 36, -36*z^2 + 36*z], 0]

In [9]:
\\(Cf. OEIS: A113424 )
PFData = HyperellipticPicardFuchs((1/2)*(x^2-(4/27)*x^6));
[-PFData[3],CheckCertificate(PFData)]

[[5*z, 27*z^2 - 9, 9*z^3 - 9*z]~, 0]

In [10]:
\\two-parameter elliptic stratum 
PFData = HyperellipticPicardFuchs((1/2)*(x^2+a*x^3+b*x^4));
[PFData[3],CheckCertificate(PFData)]

[[(1152*b^4*a^2 - 3072*b^5)*z^2 + (-18*b*a^6 + 144*b^2*a^4 + 96*b^3*a^2 - 1536*b^4)*z + (15*a^6 - 132*b*a^4 + 336*b^2*a^2 - 192*b^3), (12288*b^4*a^2 - 32768*b^5)*z^3 + (648*b*a^6 - 5184*b^2*a^4 + 15360*b^3*a^2 - 20480*b^4)*z^2 + (216*a^6 - 2016*b*a^4 + 5632*b^2*a^2 - 4096*b^3)*z + (-16*a^4 + 128*b*a^2 - 256*b^2), (6144*b^4*a^2 - 16384*b^5)*z^4 + (648*b*a^6 - 5184*b^2*a^4 + 13312*b^3*a^2 - 12288*b^4)*z^3 + (108*a^6 - 1104*b*a^4 + 3456*b^2*a^2 - 3072*b^3)*z^2 + (-16*a^4 + 128*b*a^2 - 256*b^2)*z]~, 0]

In [11]:
\\genus-degree formula: g = floor((d-1)/2) .
\\for d=3,4,5,6... ; g=1,1,2,2...  
v=[];
for(d=3,8,{
PFData = HyperellipticPicardFuchs((1/2)*(x^2+x^3+x^d));
v=concat(v,[[d,(length(PFData[3])-1)/2,CheckCertificate(PFData)]])
})
v

[[3, 1, 0], [4, 1, 0], [5, 2, 0], [6, 2, 0], [7, 3, 0], [8, 3, 0]]

In [12]:
\\apparent genus-degree formula: g' = floor(d/4) .
\\for d=4,6,8,10... ; g=1,1,2,2...  
v=[];
for(d=2,7,{
PFData = HyperellipticPicardFuchs((1/2)*(x^2+x^(2*d)));
v=concat(v,[[d,(length(PFData[3])-1)/2,CheckCertificate(PFData)]])
})
v

[[2, 1, 0], [3, 1, 0], [4, 2, 0], [5, 2, 0], [6, 3, 0], [7, 3, 0]]

Parity symmetry allows quotient decomposition of the Riemann surface into two equal pieces, how neat!